In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import datetime

In [23]:
VANTAA_Bikers=pd.read_csv('VantaaBikers_cleanDataset.csv')
VANTAA_Bikers.Date=pd.to_datetime(VANTAA_Bikers.Date,format='%Y-%m-%d')
VANTAA_Bikers=VANTAA_Bikers.set_index(['Date','Time'])
#VANTAA_Bikers.info()

ESPOO_Bikers=pd.read_csv('EspooBikers_cleanDataset.csv')
ESPOO_Bikers.Date=pd.to_datetime(ESPOO_Bikers.Date,format='%Y-%m-%d')
ESPOO_Bikers=ESPOO_Bikers.set_index(['Date','Time'])
#ESPOO_Bikers.info()

HELSINKI_Bikers=pd.read_csv('HelsinkiBikers_cleanDataset.csv')
HELSINKI_Bikers.Date=pd.to_datetime(HELSINKI_Bikers.Date,format='%Y-%m-%d')
HELSINKI_Bikers=HELSINKI_Bikers.set_index(['Date','Time'])
#HELSINKI_Bikers.info()

ALL_Bikers=HELSINKI_Bikers.copy()
ALL_Bikers=ALL_Bikers.join(VANTAA_Bikers.iloc[:,1:],how='left')
ALL_Bikers=ALL_Bikers.join(ESPOO_Bikers.iloc[:,1:],how='left')
ALL_Bikers.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51216 entries, (Timestamp('2014-01-01 00:00:00'), '00:00:00') to (Timestamp('2019-10-31 00:00:00'), '23:00:00')
Data columns (total 85 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Weekday                                        51216 non-null  object 
 1   Auroransilta                                   17330 non-null  float64
 2   Etelaesplanadi                                 50546 non-null  float64
 3   Huopalahti (station)                           39431 non-null  float64
 4   Kaisaniemi/Elaintarhanlahti                    51214 non-null  float64
 5   Kaivokatu                                      17328 non-null  float64
 6   Kulosaari bridge south                         39431 non-null  float64
 7   Kulosaaren silta po.                           39431 non-null  float64
 8   Kuusisaarentie                          

In [28]:
pyhapaivat=pd.DataFrame();
for vuosi in range(2014,2020):
    table=pd.read_html(f'https://www.webcal.fi/fi-FI/pyhat.php?y={vuosi}')
    table=table[0].iloc[:,1:4]
    table.iloc[:,2]=pd.to_datetime(table.iloc[:,2], format='%d.%m.%Y', errors='coerce')
    table.columns=['Pyhapaiva','Viikonpaiva',str(vuosi)]
    table=table.drop('Viikonpaiva',axis=1).set_index('Pyhapaiva')    
    if vuosi==2014:
        pyhapaivat=table
    else:
        pyhapaivat=pyhapaivat.join(table)

pyhapaivat_list=[date for date in pd.DatetimeIndex(pyhapaivat.iloc[:,0]).date]
for n in range(1,6):
    pyhapaivat_list.extend([date for date in pd.DatetimeIndex(pyhapaivat.iloc[:,n]).date])
pyhapaivat_set=set(pyhapaivat_list)

def onko_pyhapaiva(pdf):
    return (np.array([date in pyhapaivat_set for date in pd.DatetimeIndex(pdf.index).date]))




ALL_Bikers['pyhapaiva']=onko_pyhapaiva(ALL_Bikers.reset_index().set_index('Date'))
ALL_Bikers['nData']=ALL_Bikers.iloc[:,1:-1].T.count().astype('Int64')
columns=ALL_Bikers.columns
columns=columns[0:1].append(columns[-2:]).append(columns[1:-2])
columns
ALL_Bikers=ALL_Bikers.reindex(columns=columns)
Max_Bikers=ALL_Bikers.copy()
Max_Bikers.loc[Max_Bikers.nData==43,:]


Weekday  Suomenlahdentie (Eco-Counter)  \
Date       Time                                              
2015-05-12 15:00:00     Tue                            NaN   
           16:00:00     Tue                            NaN   
           17:00:00     Tue                            NaN   
           18:00:00     Tue                            NaN   
           19:00:00     Tue                            NaN   
...                     ...                            ...   
2016-08-12 19:00:00     Fri                            NaN   
           20:00:00     Fri                            NaN   
           21:00:00     Fri                            NaN   
           22:00:00     Fri                            NaN   
           23:00:00     Fri                            NaN   

                     Suomenlahdentie, etelainen (viacount)  \
Date       Time                                              
2015-05-12 15:00:00                                    NaN   
           16:00:00                                    NaN   
           17:00:00                                    NaN   
           18:00:00                                    NaN   
           19:00:00                                    NaN   
...                                                    ...   
2016-08-12 19:00:00                                    NaN   
           20:00:00                                    NaN   
           21:00:00                                    NaN   
           22:00:00                                    NaN   
           23:00:00                                    NaN   

                     Suomenlahdentie, pohjoinen (Viacount)  \
Date       Time                                              
2015-05-12 15:00:00                                    NaN   
           16:00:00                                    NaN   
           17:00:00                                    NaN   
           18:00:00                                    NaN   
           19:00:00                                    NaN   
...                                                    ...   
2016-08-12 19:00:00                                    NaN   
           20:00:00                                    NaN   
           21:00:00                                    NaN   
           22:00:00                                    NaN   
           23:00:00                                    NaN   

                     Tapiola,Lansituulenkuja (Eco-Counter)  \
Date       Time                                              
2015-05-12 15:00:00                                    NaN   
           16:00:00                                    NaN   
           17:00:00                                    NaN   
           18:00:00                                    NaN   
           19:00:00                                    NaN   
...                                                    ...   
2016-08-12 19:00:00                                  145.0   
           20:00:00                                   97.0   
           21:00:00                                   50.0   
           22:00:00                                   26.0   
           23:00:00                                    9.0   

                     Turuntie, Rantaradanreitti (Eco-Counter)  \
Date       Time                                                 
2015-05-12 15:00:00                                       NaN   
           16:00:00                                       NaN   
           17:00:00                                       NaN   
           18:00:00                                       NaN   
           19:00:00                                       NaN   
...                                                       ...   
2016-08-12 19:00:00                                       NaN   
           20:00:00                                       NaN   
           21:00:00                                       NaN   
           22:00:00                                       NaN   
           2